In [13]:
import pandas as pd
import sys
import_path_base = '/home/henrike/Documents/PD_AS/src/'
sys.path.insert(1, import_path_base + 'PRISM/prism/scripts/')
from PrismData import PrismParser, VariantData#VariantParser, VariantData
from Bio import AlignIO
import numpy as np

In [14]:
#read in ecoli prism data
base_dir = '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/'

In [15]:
def read_from_prism(primsfile):
	parser = PrismParser()
	dataframe = parser.read(primsfile).dataframe
	meta_data = parser.read_header(primsfile)
	return meta_data, dataframe

def write_prism(metadata, dataframe, prism_file, comment=''):
	variant_dataset = VariantData(metadata, dataframe)
	parser = PrismParser()
	parser.write(prism_file, variant_dataset, comment_lines=comment)

In [17]:
ecoli_meta_data, ecoli_df = read_from_prism(base_dir + 'results/prism_mave/prism_mave_thompsen_Ecoli_dhfr.txt')

In [18]:
#remove double mutants
ecoli_df = ecoli_df[ecoli_df['n_mut']==1]
#and introduce a WT col with WT AA + pos for searching
ecoli_df['WT'] = ecoli_df['variant'].apply(lambda x: x[0:-1])
#and rename columns
ecoli_df = ecoli_df.rename(columns={'variant':'ecoli_variant', 'n_mut':'ecoli_n_mut','aa_ref':'ecoli_aa_ref','resi':'ecoli_resi','aa_var':'ecoli_aa_var','WT':'ecoli_WT'})


In [20]:
#read in an prepare human dhfr data 
human_meta_data, human_df = read_from_prism(base_dir + 'data/prism_MAVE_001_DHFR.txt')
human_df = human_df[human_df['n_mut']==1]
human_df['WT'] = human_df['variant'].apply(lambda x: x[0:-1])
human_df = human_df.rename(columns={'variant':'human_variant', 'n_mut':'human_n_mut','aa_ref':'human_aa_ref','resi':'human_resi','aa_var':'human_aa_var','WT':'human_WT'})


In [21]:
#read in alignment
s_align = AlignIO.read(open(base_dir + 'results/tcoffee_expresso/result.clustalw_aln'), "clustal")

In [22]:
#init merged df
merge_df = pd.DataFrame(columns=list(human_df.columns)+list(ecoli_df.columns))

In [23]:
amino_acids = ['H','K','R','D','E','C','M','N','Q','S','T','A','I','L','V','F','W','Y','G','P','*']

In [24]:
#go through alignment and add rows to the merged df
human_c = 0
ecoli_c = 0
#for i in range(0, len(s_align[0].seq)):
#the current human DHFR tile only goes til 39 anyway
#full length dhfr has 187 positions
for i in range(0,187):
    if s_align[0].seq[i] != '-': human_c += 1
    if s_align[1].seq[i] != '-': ecoli_c += 1
    print('human:', s_align[0].seq[i], str(human_c), 'ecoli:', s_align[1].seq[i], str(ecoli_c))
    for aa in amino_acids:
        #append row with human:S4A and ecoli:M1A
        merge_df = merge_df.append(pd.concat([human_df.loc[human_df['human_variant'] == s_align[0].seq[i]+str(human_c)+aa].reset_index(drop=True),ecoli_df.loc[ecoli_df['ecoli_variant'] == s_align[1].seq[i]+str(ecoli_c)+aa].reset_index(drop=True)], axis = 1), ignore_index=True)
    



human: M 1 ecoli: - 0
human: V 2 ecoli: - 0
human: G 3 ecoli: - 0
human: S 4 ecoli: M 1
human: L 5 ecoli: I 2
human: N 6 ecoli: S 3
human: C 7 ecoli: L 4
human: I 8 ecoli: I 5
human: V 9 ecoli: A 6
human: A 10 ecoli: A 7
human: V 11 ecoli: L 8
human: S 12 ecoli: A 9
human: Q 13 ecoli: V 10
human: N 14 ecoli: D 11
human: M 15 ecoli: R 12
human: G 16 ecoli: V 13
human: I 17 ecoli: I 14
human: G 18 ecoli: G 15
human: K 19 ecoli: M 16
human: N 20 ecoli: E 17
human: G 21 ecoli: N 18
human: D 22 ecoli: A 19
human: L 23 ecoli: M 20
human: P 24 ecoli: P 21
human: W 25 ecoli: W 22
human: P 26 ecoli: N 23
human: P 27 ecoli: - 23
human: L 28 ecoli: L 24
human: R 29 ecoli: P 25
human: N 30 ecoli: A 26
human: E 31 ecoli: D 27
human: F 32 ecoli: L 28
human: R 33 ecoli: A 29
human: Y 34 ecoli: W 30
human: F 35 ecoli: F 31
human: Q 36 ecoli: K 32
human: R 37 ecoli: R 33
human: M 38 ecoli: N 34
human: T 39 ecoli: T 35
human: T 40 ecoli: L 36
human: T 41 ecoli: N 37
human: S 42 ecoli: - 37
human: S 43 e

In [25]:
merge_df

,human_variant,score,SE,tile,human_n_mut,human_aa_ref,human_resi,human_aa_var,human_WT,ecoli_variant,...,Lon_minus_std_dev,Lon_minus_std_err,Lon_plus_score,Lon_plus_std_dev,Lon_plus_std_err,ecoli_n_mut,ecoli_aa_ref,ecoli_resi,ecoli_aa_var,ecoli_WT
0,V2H,-0.391535,0.311642,1,1,[V],[2],[H],V2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V2K,0.206199,0.232451,1,1,[V],[2],[K],V2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V2R,0.061586,0.251602,1,1,[V],[2],[R],V2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V2D,0.121341,0.194870,1,1,[V],[2],[D],V2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V2E,0.115536,0.160239,1,1,[V],[2],[E],V2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,D187F,0.200886,0.026569,5,1,[D],[187],[F],D187,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3718,D187W,0.210491,0.050457,5,1,[D],[187],[W],D187,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3719,D187Y,0.414320,0.059295,5,1,[D],[187],[Y],D187,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3720,D187G,0.279710,0.043179,5,1,[D],[187],[G],D187,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
merge_df.to_csv(base_dir+'/data/merge_human_ecoli_fulldhfr.csv', sep = '\t', na_rep='NA')

In [27]:
#test:
merge_df2 = pd.DataFrame(columns=list(human_df.columns)+list(ecoli_df.columns))

human_c = 0
ecoli_c = 0
#for i in range(0, len(s_align[0].seq)):
for i in range(0,2):
    if s_align[0].seq[i] != '-': human_c += 1
    if s_align[1].seq[i] != '-': ecoli_c += 1
    print('human:', s_align[0].seq[i], str(human_c), 'ecoli:', s_align[1].seq[i], str(ecoli_c))
    for aa in amino_acids:
        #append row with human:S4A and ecoli:M1A
        merge_df2 = merge_df2.append(pd.concat([human_df.loc[human_df['human_variant'] == s_align[0].seq[i]+str(human_c)+aa].reset_index(drop=True),ecoli_df.loc[ecoli_df['ecoli_variant'] == s_align[1].seq[i]+str(ecoli_c)+aa].reset_index(drop=True)], axis = 1), ignore_index=True)



human: M 1 ecoli: - 0
human: V 2 ecoli: - 0


In [28]:
merge_df2
#observation:
#It skips completely empty rows (composed only of NAs) like the human:M1 line (the mave starts at pos 2 since we use a fusion protein)

,human_variant,MTX_score,MTX_SE,MTX_Cu_score,MTX_Cu_SE,37_score,37_SE,37_Cu_score,37_Cu_SE,human_n_mut,...,Lon_minus_std_dev,Lon_minus_std_err,Lon_plus_score,Lon_plus_std_dev,Lon_plus_std_err,ecoli_n_mut,ecoli_aa_ref,ecoli_resi,ecoli_aa_var,ecoli_WT
0,V2H,-0.286942,0.356365,-0.701059,0.198073,-0.319804,0.536673,0.401002,0.737741,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V2K,0.020324,0.221691,0.689756,0.490723,1.749161,1.007435,0.437940,0.281015,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V2R,0.205459,0.099615,-0.035478,0.207287,NaN,NaN,0.257201,0.378342,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V2D,-0.007734,0.744842,-0.093837,0.476336,-0.928840,0.548459,0.406764,0.776011,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V2E,-0.157151,0.148753,0.520350,0.367248,-0.158215,0.499464,-0.410229,0.428925,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,V2C,0.810929,0.684289,0.334408,1.087821,-0.314401,1.017164,0.208587,0.286785,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,V2M,0.468150,0.414734,0.371962,0.413697,0.762780,0.657686,0.163284,0.791211,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,V2N,-0.459645,0.484509,-0.361201,0.441901,-0.358981,0.322562,-0.714165,0.518612,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,V2Q,-0.140268,0.465379,0.082057,0.445276,-0.762146,0.635072,-0.147298,0.392828,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,V2S,0.020571,0.066504,-0.219469,0.168145,0.595595,0.594085,-0.116430,0.457736,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#correct indexing?

human_c = 0
ecoli_c = 0
#for i in range(0, len(s_align[0].seq)):
for i in range(0,10):
    if s_align[0].seq[i] != '-': human_c += 1
    if s_align[1].seq[i] != '-': ecoli_c += 1
    for aa in amino_acids:
        print('human:', s_align[0].seq[i]+str(human_c)+aa, 'ecoli:', s_align[1].seq[i]+str(ecoli_c)+aa)
        #append row with human:S4A and ecoli:M1A
        #merge_df = merge_df.append(pd.concat([human_df.loc[human_df['human_variant'] == s_align[0].seq[i]+str(human_c)+aa],ecoli_df.loc[ecoli_df['ecoli_variant'] == s_align[1].seq[i]+str(ecoli_c)+aa]], axis = 1), ignore_index=True)
    




human: M1H ecoli: -0H
human: M1K ecoli: -0K
human: M1R ecoli: -0R
human: M1D ecoli: -0D
human: M1E ecoli: -0E
human: M1C ecoli: -0C
human: M1M ecoli: -0M
human: M1N ecoli: -0N
human: M1Q ecoli: -0Q
human: M1S ecoli: -0S
human: M1T ecoli: -0T
human: M1A ecoli: -0A
human: M1I ecoli: -0I
human: M1L ecoli: -0L
human: M1V ecoli: -0V
human: M1F ecoli: -0F
human: M1W ecoli: -0W
human: M1Y ecoli: -0Y
human: M1G ecoli: -0G
human: M1P ecoli: -0P
human: M1* ecoli: -0*
human: V2H ecoli: -0H
human: V2K ecoli: -0K
human: V2R ecoli: -0R
human: V2D ecoli: -0D
human: V2E ecoli: -0E
human: V2C ecoli: -0C
human: V2M ecoli: -0M
human: V2N ecoli: -0N
human: V2Q ecoli: -0Q
human: V2S ecoli: -0S
human: V2T ecoli: -0T
human: V2A ecoli: -0A
human: V2I ecoli: -0I
human: V2L ecoli: -0L
human: V2V ecoli: -0V
human: V2F ecoli: -0F
human: V2W ecoli: -0W
human: V2Y ecoli: -0Y
human: V2G ecoli: -0G
human: V2P ecoli: -0P
human: V2* ecoli: -0*
human: G3H ecoli: -0H
human: G3K ecoli: -0K
human: G3R ecoli: -0R
human: G3D

In [47]:
#tmp_df = pd.concat([human_df.loc[human_df['human_variant'] == s_align[0].seq[i]+str(human_c)+aa],ecoli_df.loc[ecoli_df['ecoli_variant'] == s_align[1].seq[i]+str(ecoli_c)+aa]], axis = 1)
tmp_df = pd.concat([human_df.loc[human_df['human_variant'] == 'S4K'].reset_index(drop=True),ecoli_df.loc[ecoli_df['ecoli_variant'] == 'M1K'].reset_index(drop=True)], axis = 1)
tmp_df

,human_variant,MTX_score,MTX_SE,MTX_Cu_score,MTX_Cu_SE,37_score,37_SE,37_Cu_score,37_Cu_SE,human_n_mut,...,Lon_minus_std_dev,Lon_minus_std_err,Lon_plus_score,Lon_plus_std_dev,Lon_plus_std_err,ecoli_n_mut,ecoli_aa_ref,ecoli_resi,ecoli_aa_var,ecoli_WT
0,S4K,-0.014153,0.600629,-0.557012,0.126831,-0.119039,0.463813,-0.835569,0.278303,1,...,NaN,NaN,NaN,NaN,NaN,1,[M],[1],[K],M1


In [36]:
human_df.loc[human_df['human_variant'] == 'S4K'].reset_index(drop=True)

,human_variant,MTX_score,MTX_SE,MTX_Cu_score,MTX_Cu_SE,37_score,37_SE,37_Cu_score,37_Cu_SE,human_n_mut,human_aa_ref,human_resi,human_aa_var,human_WT
0,S4K,-0.014153,0.600629,-0.557012,0.126831,-0.119039,0.463813,-0.835569,0.278303,1,[S],[4],[K],S4


In [55]:
merge_df.loc[merge_df['human_variant'] == 'L5I']

,human_variant,MTX_score,MTX_SE,MTX_Cu_score,MTX_Cu_SE,37_score,37_SE,37_Cu_score,37_Cu_SE,human_n_mut,...,Lon_minus_std_dev,Lon_minus_std_err,Lon_plus_score,Lon_plus_std_dev,Lon_plus_std_err,ecoli_n_mut,ecoli_aa_ref,ecoli_resi,ecoli_aa_var,ecoli_WT
69,L5I,-0.696559,0.556897,-0.677255,0.284411,-1.594511,0.404429,-0.257407,0.63212,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
